In [1]:
import numpy as np
from utils import *
import matplotlib.pyplot as plt
import re
LAMBDA = 10

In [ ]:
points = 'samples/Airfoils1D_044.dat'
airfoil = np.loadtxt(points, skiprows=1)
af, R, a, b, perf, cd, CD_BL = lowestD(airfoil, Re2= 401000, lamda = LAMBDA, check_thickness=False)
name = points.split('/')[1].split('.')[0]+f'_{a}_{b}F'
np.savetxt(points.split('.')[0]+f'_{a}_{b}F'+'.dat', af, header=points.split('/')[1].split('.')[0]+f'_{a}_{b}F', comments="")
print(f'R: {R}, angle: {a}, pose: {b}, perf: {perf}, cruise cd: {cd}, launch cd: {CD_BL}')
show_airfoil(af)
print(f'intersection: {detect_intersect(af)}, thickness: {cal_thickness(af)}, thickness pose: {cal_thickness_percent(af)}, tail cross pose: {check_backpoint(af)}')

In [ ]:
perffile = 'results/Airfoils1D_simperf.log'
f = open(perffile, 'r')
p = re.compile('perf: ')
perfs = []
airfoils = []
Rs = []
for line in f:
    if p.search(line):
        perf = line.split('perf: ')[1].split(', R: ')[0]
        perf = float(perf)
        perfs.append(perf)
        R = line.split(', R: ')[1].split(', m: ')[0]
        R = float(R)
        Rs.append(R)
        airfoilfile = line.split(', path: ')[1]
        airfoils.append(airfoilfile)

perfs = np.array(perfs)
Rs = np.array(Rs)
i = np.argmax(perfs)
k = np.argmin(Rs)
# Rs = np.delete(Rs, k)
# k = np.argmin(Rs)
# Rs = np.delete(Rs, k)
# k = np.argmin(Rs)
print(airfoils[i])
print(airfoils[k], Rs[k], perfs[k])


In [ ]:
f = open(perffile, 'r')
p = re.compile(airfoils[k])
lines = []
for line in f:
    if p.search(line) is not None:
        print(line)
        pass
    else:
        lines.append(line)
f.close()
f = open(perffile, 'w')
for line in lines:
    f.write(line)
f.close()


In [ ]:

points = 'samples/Airfoils2D_049.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, -3, 0.6)
show_airfoil(af)
cd, _ = evalpreset(af)
perf, _, _ = evalperf(setflap(af, 3, 0.6))
cd, cal_thickness(af), cal_thickness_percent(af), perf, check_backpoint(af)

In [ ]:

f = open('results/Airfoils1D_simperf.log', 'r')
a = f.read()
f.close()
files = []
for name in a.split('path: ')[1:]:
    file = name.split('.dat')[0] + '.dat'
    files.append(file)

In [ ]:
for file in files:
    # points = 'samples/Airfoils1D_039.dat'
    # airfoil = np.loadtxt(points, skiprows=1)
    airfoil = np.loadtxt(file, skiprows=1)
    af, R, a, b, perf, cd, CD_BL = lowestD(airfoil, lamda = LAMBDA, check_thickness=False)
    name = file.split('/')[1].split('.')[0]+f'_{a}_{b}F'
    np.savetxt(file.split('.')[0]+f'_{a}_{b}F'+'.dat', af, header=file.split('/')[1].split('.')[0]+f'_{a}_{b}F', comments="")
    print(R, a, b, perf, cd, CD_BL)

In [ ]:
b = np.linspace(0.5, 0.6, num=5) 
a = np.linspace(-3,-1, num=5)
cdbl = 1
for pose in b:
    for alfa in a:
        afp = setupflap(airfoil, theta = alfa, pose = pose)
        cd, aa = evalpreset(afp)
        if cd < cdbl:
            pose_bl = pose
            alfa_bl = alfa
            af_bl = afp
            cdbl = cd
            print(pose_bl, alfa_bl, cdbl)

pose_bl, alfa_bl, cdbl

In [ ]:
points = 'samples/tipAirfoils1D_004.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, -3, 0.6)
show_airfoil(af)
cd, _ = evalpreset(af, Re=1.4e5)
cd, cal_thickness(af), cal_thickness_percent(af), detect_intersect(af), check_backpoint(af)

In [ ]:
points = 'samples/tipAirfoils1D_018.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, -3, 0.6)
show_airfoil(af)
cd, _ = evalpreset(af, Re=1.4e5)
cd, cal_thickness(af), cal_thickness_percent(af), detect_intersect(af), check_backpoint(af)

In [ ]:
points = 'samples/tipAirfoils2D_033.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, -3, 0.6)
show_airfoil(af)
cd, _ = evalpreset(af, Re=1.4e5)
cd, cal_thickness(af), cal_thickness_percent(af), detect_intersect(af), check_backpoint(af)

In [ ]:
points = 'samples/tipAirfoils2D_044.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, -3, 0.6)
show_airfoil(af)
cd, _ = evalpreset(af, Re=1.4e5)
cd, cal_thickness(af), cal_thickness_percent(af), detect_intersect(af), check_backpoint(af)

In [ ]:
points = 'samples/tipAirfoils1D_004.dat'
airfoil = np.loadtxt(points, skiprows=1)
airfoil = setupflap(airfoil, -3, 0.6)
def set_thickness_pose(airfoil, pose):
    lh_idx = np.argmin(airfoil[:,0])
    lh_x = airfoil[lh_idx, 0]
    # Get trailing head
    th_x = np.minimum(airfoil[0,0], airfoil[-1,0])
    # Interpolate
    f_up = interp1d(airfoil[:lh_idx+1,0], airfoil[:lh_idx+1,1])
    f_low = interp1d(airfoil[lh_idx:,0], airfoil[lh_idx:,1])
    xx = np.linspace(lh_x, th_x, num=1000)
    yy_up = f_up(xx)
    yy_low = f_low(xx)
    max_thickness_index = (yy_up-yy_low).argmax()
    p = pose*10/max_thickness_index
    xxx = np.copy(xx)
    xxx[:max_thickness_index] = xx[:max_thickness_index] * p
    xxx[max_thickness_index:] = (xx[max_thickness_index:] - xx[max_thickness_index]) * (1000 - max_thickness_index * p) / (1000 - max_thickness_index) + xx[max_thickness_index] * p
    f_up = interp1d(xxx, yy_up)
    f_low = interp1d(xxx, yy_low)
    xx = np.linspace(xxx[0], xxx[-1], num=128)
    yy_up = f_up(xx)
    yy_low = f_low(xx)
    airfoil[:128,0] = np.flip(xx)
    airfoil[:128,1] = np.flip(yy_up)
    airfoil[128:,0] = xx
    airfoil[128:,1] = yy_low
    airfoil = np.delete(airfoil, [128], axis=0)
    xhat, yhat = savgol_filter((airfoil[:,0], airfoil[:,1]), 7, 3)
    airfoil[:,0] = xhat
    airfoil[:,1] = yhat
    airfoil = interpolate(airfoil, 256, 3)
    airfoil = derotate(airfoil)
    airfoil = Normalize(airfoil)
    airfoil = interpolate(airfoil, 256, 3)
    return airfoil
airfoil = set_thickness_pose(airfoil, cal_thickness_percent(airfoil)*(1-(25-23)/25))
show_airfoil(airfoil)
cal_thickness_percent(airfoil)


In [ ]:
import os

def rewrite_polar(root):
    for path, dir, files in os.walk(root):
        for file in files:
            lines=[]
            infile = root+file
            with open(infile, 'r') as fin:
                for line in fin:
                    line = line.replace('9.000  9.000', '9.000')
                    lines.append(line)
            fin.close()
            with open(infile, "w") as fout:
                for line in lines:
                    fout.write(line)
            fout.close()

root = 'Airfoils2D_049_0.6F_20_-3/'
rewrite_polar(root)